1. 14_Data.csv를 불러와 df1으로 선언하여 아래의 미션을 수행. (해당 데이터는 TV 홈쇼핑 광고 대행사의 광고 홈쇼핑 방송 송출 데이터)
2. '방송구분'이라는 항목에서 '녹화방송' 에 해당 하는 데이터의 '프로그램명'별 '상품목표주문금액'과 '상품주문금액'의 합과 평균을 계산하시오.
3. '프로그램명'에서 가장 '상품주문금액'의 합이 큰 3가지를 확인하여, 3가지 항목에 대해 데이터를 추출한 뒤, '상품명'별 '상품주문금액'의 합을 계산하시오.
4. '방송시작시간'과 '방송종료시간'의 차이를 계산하여, '방송진행시간'이라는 항목을 선언하고, '방송진행시간'이 1시간 미만인 데이터의 개수를 확인하시오.
5. '방송시작시간'을 날짜형식으로 변환하여, '방송연도', '방송월', ‘방송일', '방송요일' 파생변수 항목을 생성하시오.
6. '방송요일'항목에서 주말과 주중을 구분하여, '주말'과 '주중'의 '상품주문수량'과 '상품주문금액'의 평균의 차이를 계산
7. '배송방식'과 '매입형태'간의 연관성이 있는지 확인하시오.
8. '방송구분'에서 '생방송'과 '녹화방송'의 매출을 비교하고자 한다. '생방송'과'녹화방송'의 '상품주문금액'의 평균의 차이가 있는지 검정
9. '상품목표주문금액'에서 '상품주문금액'의 차이를 계산해, 목표를 달성하지 못한 방송에 대해서는 0 / 목표를 달성한 방송은 1값으로 '목표달성여부' 변수를 생성
10. '목표달성여부'를 분류하는 모델을 만들고자 한다. 소요분, 방송구분, 판매단가, ARS금액, 수수료율, 방송요일, 방송월을 활용해서 목표달성여부를 예측하는 모델을 만드세요. 데이터 분할을 8 : 2로 분할하고, 특성 공학 기법으로 결측처리는 평균 및 최빈값으로 처리하고, 적절한 스케일링과 인코딩 방법을 사용하세요. 그리고 정확도, 정밀도, 재현율, F1-Score를 활용하고 모델을 평가한 후 해석하시오.

In [146]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [129]:
# 1. 14_Data.csv를 불러와 df1으로 선언하여 아래의 미션을 수행. 
# (해당 데이터는 TV 홈쇼핑 광고 대행사의 광고 홈쇼핑 방송 송출 데이터)
df1 = pd.read_csv("실습 파일/14_Data.csv")
df1

,방송Code,채널,소요분,가중분,방송구분,프로그램명,상품ID,상품명,매입과세구분,상품목표취급금액,...,상품취소수량,상품취소금액,ARS금액,매입형태,배송방식,상품소요분,상품가중분,상품방송순번,방송시작시간,방송종료시간
0,1.010036e+09,TV,50.0,52.7,녹화방송,재방_의류,10242138,[비지트인뉴욕콜렉션] 퓨어 캐시미어 롱코트,과세,65645345,...,73,9624136,1000,위탁매입,협력사배송,50.0,52.7,1,2020-01-03T00:10,2020-01-03T01:00
1,1.019126e+09,TV,60.0,38.1,녹화방송,재방_건강식품,10092003,[한삼인]순홍삼진(50ml*30포)*7박스+쇼7_2,과세,23774849,...,16,2547305,1000,위탁매입,직택배,30.0,19.1,1,2020-01-03T01:00,2020-01-03T02:00
2,1.019226e+09,TV,60.0,38.1,녹화방송,재방_건강식품,10092743,[단품_한삼인] 순홍삼진(50ml*30포)*1박스,과세,23774849,...,0,0,0,위탁매입,직택배,30.0,19.1,2,2020-01-03T01:00,2020-01-03T02:00
3,1.040337e+09,TV,60.0,19.7,재방송,재방_가공농산,10295865,이상용의 우리밀 크레마롤,과세,24568911,...,25,1080206,1000,위탁매입,협력사배송,60.0,19.7,1,2020-01-03T02:00,2020-01-03T03:00
4,1.021427e+09,TV,60.0,16.1,재방송,재방_신선수산,10113666,국내산 자숙 문어슬라이스 120g*9팩,면세,20131383,...,5,238512,1000,위탁매입,협력사배송,60.0,16.1,1,2020-01-03T03:00,2020-01-03T04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26278,1.046263e+13,TV,60.0,94.3,생방송,가전,11179512,■연말특가■제우스커브드 UHD TV 65형,과세,92367433,...,20,11783638,1000,위탁매입,협력사배송,19.9,32.6,1,2021-11-13T21:45,2021-11-13T22:45
26279,1.046263e+13,TV,60.0,94.3,생방송,가전,11179514,2019 제우스 UHD TV 75형,과세,33698544,...,4,4683636,1000,위탁매입,협력사배송,26.7,41.9,2,2021-11-13T21:45,2021-11-13T22:45
26280,1.046263e+13,TV,60.0,94.3,생방송,가전,11179504,■연말특가■제우스커브드 UHD TV 55형,과세,17202608,...,9,3667275,1000,위탁매입,협력사배송,13.4,22.7,3,2021-11-13T21:45,2021-11-13T22:45
26281,1.020263e+13,TV,60.0,91.6,재방송,재방_건강식품,11152474,[래오이경제] 흑염소진액 70ml x 120포,과세,154250864,...,27,3879990,1000,위탁매입,협력사배송,60.0,91.6,1,2021-11-13T22:45,2021-11-13T23:45


In [130]:
# 2. '방송구분'이라는 항목에서 '녹화방송' 에 해당하는 
# 데이터의 '프로그램명'별 '상품목표주문금액'과 '상품주문금액'의 합과 평균을 계산하시오.
sample = df1[df1['방송구분']=='녹화방송']
sample.pivot_table(index='프로그램명', values=['상품목표주문금액','상품주문금액'], aggfunc=['sum','mean'])

sum                     mean              
           상품목표주문금액     상품주문금액      상품목표주문금액        상품주문금액
프로그램명                                                     
게릴라 프로모션    6386241    8736100  6.386241e+06  8.736100e+06
재방_가전      30944319   41860280  1.031477e+07  1.395343e+07
재방_건강식품    97042110   56468669  2.426053e+07  1.411717e+07
재방_생활용품    31734436   38868330  6.346887e+06  7.773666e+06
재방_속옷      75912426   36617333  3.795621e+07  1.830867e+07
재방_신선농산    86213568   74268755  2.155339e+07  1.856719e+07
재방_의류     471714526  372461954  6.738779e+07  5.320885e+07
재방_이미용     75910061   64767805  1.084429e+07  9.252544e+06
재방_잡화      32719026   32174842  1.635951e+07  1.608742e+07
재방_침구      54728388   47626117  1.368210e+07  1.190653e+07

In [131]:
# 3. '프로그램명'에서 가장 '상품주문금액'의 합이 큰 3가지를 확인하여, 
# 3가지 항목에 대해 데이터를 추출한 뒤, '상품명'별 '상품주문금액'의 합을 계산하시오.
idx = df1.groupby('프로그램명')[['상품주문금액']].sum().sort_values('상품주문금액',ascending=False).head(3).index
idx

Index(['의류', '렌탈', '신선농산'], dtype='object', name='프로그램명')

In [132]:
sample = df1[df1['프로그램명'].isin(idx)]
sample.pivot_table(index='상품명', values='상품주문금액', aggfunc='sum')

,상품주문금액
상품명,
모닝애플 조생부사 세척사과 3kg*3박스,83710050
모닝애플 홍로 세척사과 3kg*3박스,260894995
(17년 햅곡)김연도 오색현미 + 옹기가마솥 세트,113245285
(17년 햅곡)김연도 오색현미 22봉,61467733
(17년 햅곡)김연도 웰빙 선물세트,40103888
...,...
황금대추 방울토마토3kg,153834944
황토방 청도 감말랭이 세트(30봉),41407760
황토방 청도 감말랭이 세트(30봉+1봉),106544036


In [133]:
# 4.'방송시작시간'과 '방송종료시간'의 차이를 계산하여, '
# 방송진행시간'이라는 항목을 선언하고, '방송진행시간'이 1시간 미만인 데이터의 개수를 확인하시오.
from datetime import timedelta
df1['방송진행시간'] = pd.to_datetime(df1['방송종료시간'])-pd.to_datetime(df1['방송시작시간'])
df1[df1['방송진행시간'] < timedelta(hours=1)].shape

(5769, 26)

In [134]:
sample = (pd.to_datetime(df1['방송종료시간'])-pd.to_datetime(df1['방송시작시간'])).dt.components
sample[(sample['days']==0)&(sample['hours']==0)].shape

(5769, 7)

In [135]:
# 5. '방송시작시간'을 날짜형식으로 변환하여, '방송연도', '방송월', ‘방송일', '방송요일' 파생변수 항목을 생성하시오.
df1['방송연도'] = pd.to_datetime(df1['방송시작시간']).dt.year
df1['방송월'] = pd.to_datetime(df1['방송시작시간']).dt.month
df1['방송일'] = pd.to_datetime(df1['방송시작시간']).dt.day
df1['방송요일'] = pd.to_datetime(df1['방송시작시간']).dt.weekday
df1[['방송시작시간','방송연도','방송월','방송일','방송요일']]

,방송시작시간,방송연도,방송월,방송일,방송요일
0,2020-01-03T00:10,2020,1,3,4
1,2020-01-03T01:00,2020,1,3,4
2,2020-01-03T01:00,2020,1,3,4
3,2020-01-03T02:00,2020,1,3,4
4,2020-01-03T03:00,2020,1,3,4
...,...,...,...,...,...
26278,2021-11-13T21:45,2021,11,13,5
26279,2021-11-13T21:45,2021,11,13,5
26280,2021-11-13T21:45,2021,11,13,5
26281,2021-11-13T22:45,2021,11,13,5


In [136]:
# 6. '방송요일'항목에서 주말과 주중을 구분하여, '주말'과 '주중'의 '상품주문수량'과 '상품주문금액'의 평균의 차이를 계산
# 가설검증
df1['주중주말'] = df1['방송요일'].replace({0:'주중',1:'주중',2:'주중',3:'주중',4:'주중',5:'주말',6:'주말'})
df1['주중주말']

0        주중
1        주중
2        주중
3        주중
4        주중
         ..
26278    주말
26279    주말
26280    주말
26281    주말
26282    주말
Name: 주중주말, Length: 26283, dtype: object

In [137]:
sample = df1.pivot_table(columns='주중주말', values=['상품주문수량','상품주문금액'])
sample['주중 주말 차이']=abs(sample['주말']-sample['주중'])
sample

주중주말,주말,주중,주중 주말 차이
상품주문금액,5.788722e+07,6.913340e+07,1.124618e+07
상품주문수량,9.595470e+02,9.428551e+02,1.669188e+01


In [148]:
# 7. '배송방식'과 '매입형태'간의 연관성이 있는지 확인하시오.
# 범주형 데이터 상관분석 이면 새로운 것 푸는거아닌가

# 귀무가설 기각, 두 그룹은 독립적이지 않으며 연관이 있다고 볼 수 있다.
cross = pd.crosstab( index=df1['배송방식'], columns=df1['매입형태'])
stats.chi2_contingency(cross)

# 신뢰도가 있다는 기대빈도 각 셀의 값들이 5 이상으로 되어있다면 신뢰가 간다는 것


Chi2ContingencyResult(statistic=26256.193038804344, pvalue=0.0, dof=2, expected_freq=array([[  980.45428604,    39.54571396],
       [  957.38477343,    38.61522657],
       [23326.16094053,   940.83905947]]))

In [139]:
# 8. '방송구분'에서 '생방송'과 '녹화방송'의 매출을 비교하고자 한다. 
# '생방송'과'녹화방송'의 '상품주문금액'의 평균의 차이가 있는지 검정
sample1 = df1[df1['방송구분']=='생방송']['상품주문금액']
sample2 = df1[df1['방송구분']=='녹화방송']['상품주문금액']

# 귀무가설 기각, 정규성을 따르지 않는다
# 정규성검정할 때 결측치를 처리하고 해야 한다.
stats.normaltest(sample1),stats.normaltest(sample1)

(NormaltestResult(statistic=17929.951096292796, pvalue=0.0),
 NormaltestResult(statistic=17929.951096292796, pvalue=0.0))

In [140]:
# 귀무가설 기각, 대푯값의 차이가 있다고 볼 수 있다.
stats.ranksums(sample1, sample2)

RanksumsResult(statistic=5.897121686301845, pvalue=3.698970792035826e-09)

In [149]:
# 9. '상품목표주문금액'에서 '상품주문금액'의 차이를 계산해, 
# 목표를 달성하지 못한 방송에 대해서는 0 / 
# 목표를 달성한 방송은 1값으로 '목표달성여부' 변수를 생성

df1['목표달성여부'] = 1

# 미달성
idx = df1[(df1['상품주문금액'] - df1['상품목표주문금액'])<0].index
df1.iloc[idx, -1] = 0
df1.head(3)

print(idx)

Index([    0,     2,     3,     4,     6,     7,     8,     9,    11,    12,
       ...
       26267, 26269, 26271, 26273, 26274, 26275, 26277, 26278, 26281, 26282],
      dtype='int64', length=16333)


In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report

In [147]:
# 10.'목표달성여부'를 분류하는 모델을 만들고자 한다. 
# 소요분, 방송구분, 판매단가, ARS금액, 수수료율, 방송요일, 방송월을 활용해서 목표달성여부를 예측하는 모델을 만드세요. 
# 데이터 분할을 8 : 2로 분할하고, 특성 공학 기법으로 결측처리는 평균 및 최빈값으로 처리하고, 
# 적절한 스케일링과 인코딩 방법을 사용하세요. 그리고 정확도, 정밀도, 재현율, F1-Score를 활용하고 모델을 평가한 후 해석하시오.
X = df1[['소요분','방송구분','판매단가','ARS금액','수수료율','방송요일','방송월']]
y = df1['목표달성여부']

train_x, test_x, train_y, test_y = train_test_split(X,y,test_size = 0.2)

# 숫자형 리스트 생성
numeric_list = X.describe().columns

# 범주형 리스트를 생성
category_list = X.describe(include='object').columns

# 숫자형이 들어왔을 때 결측치 처리
numeric_pipe = make_pipeline(SimpleImputer(strategy = 'mean'), StandardScaler())

# 범주형이 들어왔을 때 결측치 처리
category_pipe = make_pipeline(SimpleImputer(strategy = 'most_frequent'), OneHotEncoder())

# pipe를 합쳐주고
pipe = make_column_transformer((numeric_pipe, numeric_list),(category_pipe, category_list))

# 의사결정트리를 사용
model_pipe = make_pipeline(pipe, DecisionTreeClassifier())

# 데이터 학습
model_pipe.fit(train_x, train_y)

# 데이터 분석 결과
report = classification_report(test_y, model_pipe.predict(test_x))
print(report)

              precision    recall  f1-score   support

           0       0.69      0.71      0.70      3290
           1       0.50      0.47      0.48      1967

    accuracy                           0.62      5257
   macro avg       0.59      0.59      0.59      5257
weighted avg       0.62      0.62      0.62      5257



#### classification_report를 해석해보자
y의 값이 이진분류임으로, 0과 1로 구분되어 있음              
0을 기준으로 봤을 때,precision, recall, f1-score 값들을 확인하는 것 (0이 참, 예측값)

우린,목표달성여부가 중요함으로 지금 1을 기준으로 봐야함.      
그리고 의사결정나무트리의 경우는 recall 값이 중요함       

recall 값이 0.47임으로 모델의 성능이 좋지 않음

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
X= df1[['소요분','방송구분','판매단가','ARS금액','수수료율','방송요일','방송월']]
Y = df1['목표달성여부']

encoder = LabelEncoder()
X['방송구분'] = encoder.fit_transform(X['방송구분'])